In [1]:
# Load helper code
!git clone https://github.com/edgi-govdata-archiving/ECHO_modules.git &>/dev/null;
import pandas
import ipywidgets as widgets

In [2]:
# Make selections about programs, active facilities, and NAICS codes
from backend_scripts.programs_facilities_codes import make_selections
make_selections()


SelectMultiple(description='Programs', options=('NPDES_FLAG', 'AIR_FLAG', 'RCRA_FLAG'), value=())

Checkbox(value=False, description='Only active facilities?')

Text(value='', description='NAICS codes', placeholder='Enter NAICS codes here, separated with a comma')

[SelectMultiple(description='Programs', options=('NPDES_FLAG', 'AIR_FLAG', 'RCRA_FLAG'), value=()),
 Checkbox(value=False, description='Only active facilities?'),
 Text(value='', description='NAICS codes', placeholder='Enter NAICS codes here, separated with a comma')]

In [3]:
# Create the SQL query string
# Filter by programs
from backend_scripts.query_and_filter import query
query(make_selections())



SelectMultiple(description='Programs', options=('NPDES_FLAG', 'AIR_FLAG', 'RCRA_FLAG'), value=())

Checkbox(value=False, description='Only active facilities?')

Text(value='', description='NAICS codes', placeholder='Enter NAICS codes here, separated with a comma')

(() and ("FAC_NAICS_CODES" like ' %%%%%%%' or "FAC_NAICS_CODES" like '%%%%%%'))


In [4]:
# Access the ECHO_EXPORTER table
# Output will be the matching tracked by EPA
from ECHO_modules.get_data import get_echo_data

sql = 'select "FAC_NAME", "CWA_NAICS", "CAA_NAICS", "RCRA_NAICS", "FAC_NAICS_CODES", "FAC_LAT", "FAC_LONG" from "ECHO_EXPORTER" where ' + sql_string
try:
  echo = get_echo_data( sql )
except pandas.errors.EmptyDataError:
  print("\nThere are no records.\n")
  
echo

NameError: name 'sql_string' is not defined

In [ ]:
# Export the data to the "Files" side panel for download
echo.to_csv("data.csv")

In [ ]:
# Map the facilities
import folium
from folium.plugins import MarkerCluster
import math

map = folium.Map()
marker_cluster = MarkerCluster().add_to(map)

for index, row in echo.iterrows():
  if math.isnan(row["FAC_LAT"]) or math.isnan(row["FAC_LONG"]): #ignore facilities with no spatial data
    pass
  else:
    marker = folium.Marker(
        location = [float(row["FAC_LAT"]), float(row["FAC_LONG"])],
        popup = row["FAC_NAME"]
    ).add_to(marker_cluster)

map